In [28]:
import pandas as pd
import seaborn as sns

## Definição do problema


**Cartola FC** é um Jogo eletrônico de futebol estilo Fantasy Games, ou seja, é um jogo fictício no qual as pessoas montam seus times com jogadores de futebol da vida real, que já conta com mais de 5 milhões de usuários cadastrados. 

### Regras e objetivos do jogo 
No início de cada temporada, cada jogador ou treinador profissional do Campeonato Brasileiro recebe um valor virtual em cartoletas (moeda corrente para transações no jogo) e, de acordo com sua performance medida em pontos no decorrer das rodadas, tem seu preço valorizado ou desvalorizado.

O usuário monta seu time inicial adquirindo onze jogadores e um treinador, dentro do limite de seu orçamento (que, inicialmente, é de **C$ 100**).

O jogo, que é baseado no Campeonato Brasileiro de Futebol, **consiste em realizar a compra e venda fictícias de jogadores, sendo que estes podem ser valorizados ou desvalorizados, conforme sua atuação em cada jogo real do campeonato**.

**O objetivo é obter mais pontos que os adversários**. A pontuação varia de acordo com os resultados obtidos pelos atletas pertencentes ao time fictício montado pelo usuário. A pontuação é somada e poucas horas após o término dos jogos da rodada é publicado o desempenho de cada um dos times. 

### Definição do Problema

Dados o histórico de dados do jogo CartolaFC, podemos dar *scores* para os jogadores, em termos de potencial de ganhos em Cartoletas?

In [9]:
%ls data/2017

2017_jogadores.csv   2017_scouts_raw.csv  2017_times.csv
2017_partidas.csv    2017_tabela.csv      README.md


## Coleta de dados


In [27]:
j2017 = pd.read_csv('./data/2017/2017_jogadores.csv')
s2017 = pd.read_csv('./data/2017/2017_scouts_raw.csv')
s2017

,Unnamed: 0,A,CA,CV,DD,DP,FC,FD,FF,FS,...,atletas.foto,atletas.jogos_num,atletas.media_num,atletas.nome,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,Rodada,atletas.status_id,atletas.variacao_num
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/24/b3a08...,0,0.00,Juan Silveira dos Santos,0.00,zag,5.00,0,Nulo,0.00
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/18/83977...,0,0.00,José Roberto da Silva Júnior,0.00,lat,8.00,0,Provável,0.00
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/23/51bd7...,0,0.00,Paulo Autuori,0.00,tec,10.00,0,Provável,0.00
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/02/21/f5264...,0,0.00,Augusto Sérgio Ferreira,0.00,tec,4.00,0,Provável,0.00
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/05/01/e3a7e...,0,0.00,Ney Franco da Silveira Júnior,0.00,tec,4.00,0,Provável,0.00
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2016/08/09/24673...,0,0.00,Luis Antônio Venker de Menezes,0.00,tec,12.00,0,Provável,0.00
6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/25/170c2...,0,0.00,Gilson Kleina,0.00,tec,4.00,0,Provável,0.00
7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/28/b7b4b...,0,0.00,Marcelo Ribeiro Cabo,0.00,tec,2.00,0,Provável,0.00
8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/23/755b6...,0,0.00,Paulo André Cren Benini,0.00,zag,6.00,0,Nulo,0.00
9,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://s.glbimg.com/es/sde/f/2017/04/18/fa3a7...,0,0.00,Michel Fernandes Bastos,0.00,mei,8.00,0,Nulo,0.00



## Análise de dados

## Estatística

## Modelagem